## Experiment Setup

### Random seed / PyTorch / CUDA related

In [1]:
import time
import datetime
import os
import sys
import itertools

# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    !pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 682

### Third party libraries (NumPy, PyTorch, Pyro)

In [2]:
# Third party libraries import
import numpy as np
import torch
import pyro
import matplotlib.pyplot as plt

# Print version information
print("Python Version: " + sys.version)
print("NumPy Version: " + np.__version__)
print("PyTorch Version: " + torch.__version__)
print("Pyro Version: " + pyro.__version__)

Python Version: 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
NumPy Version: 1.17.2
PyTorch Version: 1.3.1
Pyro Version: 1.0.0


In [3]:
# More imports...
from torch import nn, optim
from torch.utils.data import random_split, DataLoader, RandomSampler
import torchvision
import torchvision.transforms as transforms
from pyro.infer import SVI, Trace_ELBO, HMC, MCMC

# Import model and dataset classes from ronald_bdl
from ronald_bdl import models, datasets
from ronald_bdl.models import utils

# pyplot setting
%matplotlib inline

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')

    torch.backends.cudnn.deterministic = True
    
    # Disable 'benchmark' mode
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory = True # Faster Host to GPU copies with page-locked memory

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

### Variable settings

#### Data prep

In [4]:
# Dataset to use
uci_dataset_name = 'bostonHousing'

# Set the proportion of the original dataset to be available as a whole
subset_proportions = [0.01, 0.1, 1]

# Proportion of the dataset to be used for training
dataset_train_size = 0.9

# Number of dataset splits
n_splits = 10

#### NN settings

In [5]:
# Fully-Connected network hidden layers
network_hidden_dims = [25, 50, 100]
network_hidden_layers = [1, 3, 5]

# Dropout
dropout_rates = [0.1, 0.3, 0.5]

# Length-scale
length_scale_values = [1e-2]

# Model Precision
tau_values = [0.1, 0.15, 0.2]


### Training setup

In [6]:
# Epochs
n_epoch_values = [40, 400, 4000]

# Optimizer learning rate
learning_rate = 0.001 # PyTorch default value is 0.001

# Training data batch sizes
if uci_dataset_name in ('protein-tertiary-structure'): # Note: 45730 rows originally
    n_training_batch = 256
elif uci_dataset_name in ('yacht', 'bostonHousing'):
    n_training_batch = 32

# Number of test predictions (for each data point)
n_prediction = 1000

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

# Test start time
test_start_time = datetime.datetime.today().strftime('%Y%m%d%H%M')

## Train the network

In [7]:
for subset_prop in subset_proportions:

    """
    Dataset multiple splits prep
    """

    # Prepare new subset of the original dataset
    subset = datasets.UCIDatasets(
        uci_dataset_name, root_dir='./datasets_files', 
        limit_size=subset_prop,
        transform=lambda X, mean, std: (X - mean) / std,
        target_transform=lambda y, mean, std: (y - mean) / std,
        download=True)

    # Determine sizes of training and testing set
    train_size = int(dataset_train_size * len(subset))
    test_size = len(subset) - train_size
    
    # Print the size of the subset
    print("subset size = " + str((len(subset), subset.n_features)))
    print("training set size = %d" % train_size)
    print("test set size = %d" % test_size)
    print()
    
    # Prepare multiple sets of random train-test splits 
    # to test the parameter combination
    subset_splits = []

    for _ in range(n_splits):
        train, test = random_split(subset, lengths=[train_size, test_size])
        subset_splits.append((train, test))

    # With all the splits, test out each combination of hyperparameters
    for hidden_dim, n_hidden, dropout_rate, length_scale, tau in itertools.product(
        network_hidden_dims, network_hidden_layers,
        dropout_rates, length_scale_values, tau_values,
    ):
        # Reset the random number generator for each method (to produce identical results)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
        pyro.set_rng_seed(random_seed)

        # Print parameter combinations being tested
        print(
            "subset %f, n_hidden %d, hidden_dim %d, dropout_rate %f, length_scale %f, tau %f"
            % (subset_prop, n_hidden, hidden_dim, dropout_rate, length_scale, tau))   

        # Try learning with different splits
        for s, (train, test) in enumerate(subset_splits):
            train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)

            # Prepare network
            network = models.FCNet(
              input_dim=subset.n_features, 
              output_dim=subset.n_targets,
              hidden_dim=hidden_dim,
              n_hidden=n_hidden,
              dropout_rate=dropout_rate,
              dropout_type='bernoulli',
            )

            # Send the whole model to the selected torch.device
            network.to(torch_device)

            # Model to train mode
            network.train()

            # Adam optimizer
            # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam

            # NOTE: Need to set L2 regularization from here
            reg_strength = utils.reg_strength(dropout_rate, length_scale, train_size, tau)

            print('reg_strength = ' + str(reg_strength))

            optimizer = optim.Adam(
                network.parameters(),
                lr=learning_rate,
                weight_decay=reg_strength, # L2 regularization
            )

            accumulated_epochs = 0

            for n_epoch in n_epoch_values:

                """
                Training
                """

                print('Training with split %d' % s)
                print("Training for %d epochs total." % n_epoch)

                # Record training start time
                tic = time.time()

                for epoch in range(n_epoch-accumulated_epochs): # loop over the dataset multiple times
                    # Mini-batches
                    for data in train_loader:
                        # get the inputs; data is a list of [inputs, labels]
                        inputs, targets = data

                        # Store the batch to torch_device's memory
                        inputs = inputs.to(torch_device)
                        targets = targets.to(torch_device)

                        # zero the parameter gradients
                        optimizer.zero_grad()

                        # forward + backward + optimize
                        outputs = network(inputs)

                        loss = objective(outputs, targets)
                        loss.backward()

                        optimizer.step()

                    print("epoch %d loss = %f" % (epoch, loss.item()))
                
                # Record training end time
                toc = time.time()
                
                # Track the number of epochs done so far
                accumulated_epochs += (n_epoch - accumulated_epochs)
                
                # Report the total training time
                print("training time = " + str(toc - tic) + " seconds")
                
                # Report the final loss
                print("final loss = %f" % (loss.item()))
                print()

                """
                Testing
                """

                # Model to eval mode
                network.eval()

                # Get the test data
                inputs, targets = test.dataset[test.indices]

                # Store the batch to torch_device's memory
                inputs = inputs.to(torch_device)
                targets = targets.to(torch_device)

                # Record testing start time
                tic_testing = time.time()

                _, mean, var, metrics = network.predict_dist(
                    inputs, n_prediction,
                    y_test=targets, reg_strength=reg_strength, train_size=train_size)

                # Record testing end time
                toc_testing = time.time()
                
                # Report the total testing time
                print("testing time = " + str(toc_testing - tic_testing) + " seconds")

                # Record all the scores to the score files
                """
                Results file storage
                """

                # Create directory to store results for the current test configuration
                test_results_path = os.path.join(
                    './test_results',
                    'error_convergence_1',
                    uci_dataset_name,
                    test_start_time,
                    (
                        str(subset_prop) 
                        + '_' + str(hidden_dim) 
                        + '_' + str(n_hidden) 
                        + '_' + str(dropout_rate) 
                        + '_' + str(length_scale)
                        + '_' + str(tau)
                        + '_' + str(n_epoch)),
                )

                os.makedirs(test_results_path, exist_ok=True)

                test_results_rmse_mc_path = os.path.join(
                    test_results_path,
                    "rmse_mc.txt"
                )

                test_results_lls_mc_path = os.path.join(
                    test_results_path,
                    "lls_mc.txt"
                )

                test_results_rmse_non_mc_path = os.path.join(
                    test_results_path,
                    "rmse_non_mc.txt"
                ) 

                if len(metrics) > 0:
                    for key, value in metrics.items():
                        print(str(key) + " = " + str(value))

                        if key == 'rmse_mc':
                            with open(test_results_rmse_mc_path, 'a+') as rmse_mc_file:
                                rmse_mc_file.write('%d %f \n' % (s, value))

                        elif key == 'test_ll_mc':
                            with open(test_results_lls_mc_path, 'a+') as lls_mc_file:
                                lls_mc_file.write('%d %f \n' % (s, value))

                        elif key == 'rmse_non_mc':
                            with open(test_results_rmse_non_mc_path, 'a+') as rmse_non_mc_file:
                                rmse_non_mc_file.write('%d %f \n' % (s, value))
                print()

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51

subset 1.000000, n_hidden 1, hidden_dim 25, dropout_rate 0.100000, length_scale 0.010000, tau 0.100000
reg_strength = tensor(9.8901e-07)
Training with split 0
Training for 40 epochs total.
epoch 0 loss = 0.967787
epoch 1 loss = 0.683742
epoch 2 loss = 0.265110
epoch 3 loss = 0.238347
epoch 4 loss = 0.080469
epoch 5 loss = 0.069511
epoch 6 loss = 0.184735
epoch 7 loss = 0.195081
epoch 8 loss = 0.426098
epoch 9 loss = 0.046724
epoch 10 loss = 0.424686
epoch 11 loss = 0.142798
epoch 12 loss = 0.171792
epoch 13 loss = 0.169145
epoch 14 loss = 0.125385
epoch 15 loss = 0.099088
epoch 16 loss = 0.128437
epoch 17 loss = 0.455944
epoch 18 loss = 0.075536
epoch 19 loss = 0.666178
epoch 20 loss = 0.100946
epoch 21 loss = 0.080837
epoch 22 loss = 0.432725
epoch 23 loss = 0.292580
epoch 24 loss = 0.224616
epoch 25 loss = 0.045227
epoch 26 loss = 0.06667

/Users/bseoh/Downloads/bayesian-dl-experiments-master/ronald_bdl/models/fcnet.py:183: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  kwargs['reg_strength'], dtype=torch.float)
/Users/bseoh/Downloads/bayesian-dl-experiments-master/ronald_bdl/models/fcnet.py:200: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  / (2 * train_size * reg_strength))


epoch 1 loss = 0.037463
epoch 2 loss = 0.030788
epoch 3 loss = 0.031330
epoch 4 loss = 0.030595
epoch 5 loss = 0.029684
epoch 6 loss = 0.028838
epoch 7 loss = 0.027840
epoch 8 loss = 0.027061
epoch 9 loss = 0.026121
epoch 10 loss = 0.025340
epoch 11 loss = 0.024529
epoch 12 loss = 0.023891
epoch 13 loss = 0.023231
epoch 14 loss = 0.022495
epoch 15 loss = 0.022082
epoch 16 loss = 0.021388
epoch 17 loss = 0.020485
epoch 18 loss = 0.020341
epoch 19 loss = 0.020102
epoch 20 loss = 0.019657
epoch 21 loss = 0.019074
epoch 22 loss = 0.018950
epoch 23 loss = 0.018070
epoch 24 loss = 0.017683
epoch 25 loss = 0.017387
epoch 26 loss = 0.016789
epoch 27 loss = 0.016343
epoch 28 loss = 0.016109
epoch 29 loss = 0.015653
epoch 30 loss = 0.015077
epoch 31 loss = 0.014510
epoch 32 loss = 0.014369
epoch 33 loss = 0.013473
epoch 34 loss = 0.014036
epoch 35 loss = 0.013250
epoch 36 loss = 0.012931
epoch 37 loss = 0.012982
epoch 38 loss = 0.012584
epoch 39 loss = 0.012506
epoch 40 loss = 0.012120
epoch 41 

epoch 322 loss = 0.000672
epoch 323 loss = 0.001274
epoch 324 loss = 0.001445
epoch 325 loss = 0.001126
epoch 326 loss = 0.000556
epoch 327 loss = 0.000858
epoch 328 loss = 0.000842
epoch 329 loss = 0.000715
epoch 330 loss = 0.000548
epoch 331 loss = 0.001000
epoch 332 loss = 0.001594
epoch 333 loss = 0.001275
epoch 334 loss = 0.000619
epoch 335 loss = 0.000676
epoch 336 loss = 0.000685
epoch 337 loss = 0.000750
epoch 338 loss = 0.000576
epoch 339 loss = 0.000434
epoch 340 loss = 0.000851
epoch 341 loss = 0.001200
epoch 342 loss = 0.000956
epoch 343 loss = 0.000587
epoch 344 loss = 0.000355
epoch 345 loss = 0.000632
epoch 346 loss = 0.000594
epoch 347 loss = 0.000595
epoch 348 loss = 0.000405
epoch 349 loss = 0.000850
epoch 350 loss = 0.000764
epoch 351 loss = 0.000913
epoch 352 loss = 0.000653
epoch 353 loss = 0.000531
epoch 354 loss = 0.000530
epoch 355 loss = 0.000412
epoch 356 loss = 0.000378
epoch 357 loss = 0.000755
epoch 358 loss = 0.000602
epoch 359 loss = 0.000506
training tim

epoch 274 loss = 0.000443
epoch 275 loss = 0.000652
epoch 276 loss = 0.000335
epoch 277 loss = 0.000626
epoch 278 loss = 0.000525
epoch 279 loss = 0.000914
epoch 280 loss = 0.000519
epoch 281 loss = 0.000677
epoch 282 loss = 0.000721
epoch 283 loss = 0.000724
epoch 284 loss = 0.000612
epoch 285 loss = 0.000539
epoch 286 loss = 0.000708
epoch 287 loss = 0.000366
epoch 288 loss = 0.000670
epoch 289 loss = 0.000639
epoch 290 loss = 0.000729
epoch 291 loss = 0.000616
epoch 292 loss = 0.000540
epoch 293 loss = 0.000481
epoch 294 loss = 0.000684
epoch 295 loss = 0.000853
epoch 296 loss = 0.000478
epoch 297 loss = 0.000524
epoch 298 loss = 0.000980
epoch 299 loss = 0.000745
epoch 300 loss = 0.000597
epoch 301 loss = 0.000486
epoch 302 loss = 0.000850
epoch 303 loss = 0.000663
epoch 304 loss = 0.000750
epoch 305 loss = 0.000543
epoch 306 loss = 0.000918
epoch 307 loss = 0.000497
epoch 308 loss = 0.000540
epoch 309 loss = 0.000504
epoch 310 loss = 0.000747
epoch 311 loss = 0.000686
epoch 312 lo

...

epoch 677 loss = 0.000271
epoch 678 loss = 0.000327
epoch 679 loss = 0.000193
epoch 680 loss = 0.000197
epoch 681 loss = 0.000315
epoch 682 loss = 0.000097
epoch 683 loss = 0.000077
epoch 684 loss = 0.000241
epoch 685 loss = 0.000191
epoch 686 loss = 0.000217
epoch 687 loss = 0.000213
epoch 688 loss = 0.000266
epoch 689 loss = 0.000147
epoch 690 loss = 0.000354
epoch 691 loss = 0.000090
epoch 692 loss = 0.000789
epoch 693 loss = 0.000666
epoch 694 loss = 0.000630
epoch 695 loss = 0.000446
epoch 696 loss = 0.000601
epoch 697 loss = 0.000840
epoch 698 loss = 0.001096
epoch 699 loss = 0.000634
epoch 700 loss = 0.000464
epoch 701 loss = 0.001300
epoch 702 loss = 0.001045
epoch 703 loss = 0.000834
epoch 704 loss = 0.001499
epoch 705 loss = 0.000930
epoch 706 loss = 0.000607
epoch 707 loss = 0.000718
epoch 708 loss = 0.000334
epoch 709 loss = 0.000108
epoch 710 loss = 0.000401
epoch 711 loss = 0.000174
epoch 712 loss = 0.000187
epoch 713 loss = 0.000260
epoch 714 loss = 0.000064
epoch 715 lo

epoch 993 loss = 0.000202
epoch 994 loss = 0.000339
epoch 995 loss = 0.000089
epoch 996 loss = 0.000143
epoch 997 loss = 0.000372
epoch 998 loss = 0.000204
epoch 999 loss = 0.000227
epoch 1000 loss = 0.000469
epoch 1001 loss = 0.000321
epoch 1002 loss = 0.000750
epoch 1003 loss = 0.000176
epoch 1004 loss = 0.000288
epoch 1005 loss = 0.000380
epoch 1006 loss = 0.000532
epoch 1007 loss = 0.000547
epoch 1008 loss = 0.000499
epoch 1009 loss = 0.000592
epoch 1010 loss = 0.000713
epoch 1011 loss = 0.000839
epoch 1012 loss = 0.000864
epoch 1013 loss = 0.000764
epoch 1014 loss = 0.000728
epoch 1015 loss = 0.000668
epoch 1016 loss = 0.000562
epoch 1017 loss = 0.000718
epoch 1018 loss = 0.000508
epoch 1019 loss = 0.000752
epoch 1020 loss = 0.000517
epoch 1021 loss = 0.000554
epoch 1022 loss = 0.000534
epoch 1023 loss = 0.000359
epoch 1024 loss = 0.000642
epoch 1025 loss = 0.000418
epoch 1026 loss = 0.000693
epoch 1027 loss = 0.000374
epoch 1028 loss = 0.000362
epoch 1029 loss = 0.000111
epoch 10

epoch 1300 loss = 0.000755
epoch 1301 loss = 0.000482
epoch 1302 loss = 0.000267
epoch 1303 loss = 0.000498
epoch 1304 loss = 0.000468
epoch 1305 loss = 0.000271
epoch 1306 loss = 0.000572
epoch 1307 loss = 0.000553
epoch 1308 loss = 0.000576
epoch 1309 loss = 0.000406
epoch 1310 loss = 0.000488
epoch 1311 loss = 0.000574
epoch 1312 loss = 0.000438
epoch 1313 loss = 0.000357
epoch 1314 loss = 0.000285
epoch 1315 loss = 0.000288
epoch 1316 loss = 0.000177
epoch 1317 loss = 0.000186
epoch 1318 loss = 0.000210
epoch 1319 loss = 0.000266
epoch 1320 loss = 0.000097
epoch 1321 loss = 0.000122
epoch 1322 loss = 0.000277
epoch 1323 loss = 0.000154
epoch 1324 loss = 0.000223
epoch 1325 loss = 0.000241
epoch 1326 loss = 0.000279
epoch 1327 loss = 0.000181
epoch 1328 loss = 0.000216
epoch 1329 loss = 0.000236
epoch 1330 loss = 0.000257
epoch 1331 loss = 0.000106
epoch 1332 loss = 0.000246
epoch 1333 loss = 0.000363
epoch 1334 loss = 0.000151
epoch 1335 loss = 0.000217
epoch 1336 loss = 0.000294
e

epoch 1605 loss = 0.000984
epoch 1606 loss = 0.000742
epoch 1607 loss = 0.000514
epoch 1608 loss = 0.000671
epoch 1609 loss = 0.000432
epoch 1610 loss = 0.000221
epoch 1611 loss = 0.000120
epoch 1612 loss = 0.000184
epoch 1613 loss = 0.000461
epoch 1614 loss = 0.000606
epoch 1615 loss = 0.000936
epoch 1616 loss = 0.001055
epoch 1617 loss = 0.001227
epoch 1618 loss = 0.001316
epoch 1619 loss = 0.001653
epoch 1620 loss = 0.001503
epoch 1621 loss = 0.001466
epoch 1622 loss = 0.001247
epoch 1623 loss = 0.001237
epoch 1624 loss = 0.001327
epoch 1625 loss = 0.001248
epoch 1626 loss = 0.001289
epoch 1627 loss = 0.001330
epoch 1628 loss = 0.001662
epoch 1629 loss = 0.001771
epoch 1630 loss = 0.001742
epoch 1631 loss = 0.001592
epoch 1632 loss = 0.001008
epoch 1633 loss = 0.000715
epoch 1634 loss = 0.000494
epoch 1635 loss = 0.000392
epoch 1636 loss = 0.000439
epoch 1637 loss = 0.000408
epoch 1638 loss = 0.000504
epoch 1639 loss = 0.000321
epoch 1640 loss = 0.000272
epoch 1641 loss = 0.000167
e

KeyboardInterrupt: 

## Results visualization

In [ ]:
# experiment_root_directory = os.path.join(
#     './test_results',
#     'error_convergence_1',
#     uci_dataset_name,
#     test_start_time,
# )

# for subset_prop, dropout_rate, reg_strength, n_epoch in itertools.product(
#     subset_proportions, dropout_rates, reg_strengths, n_epochs
# ):
#     for metric_name in ['lls_mc', 'rmse_mc', 'rmse_non_mc']:
#         figure_file_name = (
#             str(subset_prop)
#             + '_' + str(dropout_rate)
#             + '_' + str(reg_strength)
#             + '_' + str(n_epoch)
#             + '_' + metric_name + '.png'
#         )
        
#         figure_title = (
#             metric_name 
#             + (' subset %f, dropout rate = %f, reg strength = %f, n_epoch = %d' % (subset_prop, dropout_rate, reg_strength, n_epoch))
#         )

#         fig, ax = plt.subplots(len(network_hidden_dims), len(network_hidden_layers))

#         fig.tight_layout(pad=2, rect=[0, 0.00, 2, 2])    

#         plt_subplot_current_row = 0
        
#         for hidden_dim in network_hidden_dims:
#             plt_subplot_current_column = 0

#             for n_layer in network_hidden_layers:
#                 # Open the score file
#                 score_file_path = os.path.join(
#                     experiment_root_directory,
#                     (
#                         str(subset_prop) 
#                         + '_' + str(hidden_dim)
#                         + '_' + str(n_layer) 
#                         + '_' + str(dropout_rate) 
#                         + '_' + str(reg_strength)
#                         + '_' + str(n_epoch)
#                     ),
#                     metric_name + '.txt',
#                 )

#                 scores = np.loadtxt(score_file_path).T
                
#                 #if metric_name in ('rmse_mc', 'rmse_non_mc'):
#                 #    ax[plt_subplot_current_row, plt_subplot_current_column].set_ylim([0, 20])
#                 #elif metric_name == 'lls_mc':
#                 #    ax[plt_subplot_current_row, plt_subplot_current_column].set_ylim([-20, 0])

#                 ax[plt_subplot_current_row, plt_subplot_current_column].scatter(scores[0], scores[1])

#                 ax[plt_subplot_current_row, plt_subplot_current_column].set_title(str((hidden_dim, n_layer)))

#                 plt_subplot_current_column += 1

#             plt_subplot_current_row += 1

#         fig.suptitle(figure_title, y=2.05)
        
#         plt.savefig(
#             os.path.join(experiment_root_directory, figure_file_name),
#             dpi=600,
#             bbox_inches='tight',
#         )
        
#         plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

experiment_root_directory = os.path.join(
    './test_results',
    'error_convergence_1',
    uci_dataset_name,
    test_start_time,
)

for subset_prop, dropout_rate, length_scale, tau, n_epoch in itertools.product(
    subset_proportions, dropout_rates, length_scale_values, tau_values, n_epochs
):
    for metric_name in ['lls_mc', 'rmse_mc', 'rmse_non_mc']:
        figure_file_name = (
            '3d_' + str(subset_prop)
            + '_' + str(dropout_rate)
            + '_' + str(length_scale)
            + '_' + str(tau)
            + '_' + str(n_epoch)
            + '_' + metric_name + '.png'
        )
        
        figure_title = (
            metric_name 
            + (' subset %f, dropout rate = %f, length_scale %f, tau %f, n_epoch = %d' 
               % (subset_prop, dropout_rate, length_scale, tau, n_epoch))
        )

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        fig.tight_layout(pad=2, rect=[0, 0.00, 2, 2])
        
        hidden_dims_data = []
        hidden_layers_data = []
        scores_data = []
        
        for hidden_dim in network_hidden_dims:
            for n_layer in network_hidden_layers:
                # Open the score file
                score_file_path = os.path.join(
                    experiment_root_directory,
                    (
                        str(subset_prop) 
                        + '_' + str(hidden_dim)
                        + '_' + str(n_layer) 
                        + '_' + str(dropout_rate) 
                        + '_' + str(length_scale)
                        + '_' + str(tau)
                        + '_' + str(n_epoch)
                    ),
                    metric_name + '.txt',
                )

                scores = np.loadtxt(score_file_path).T
                
                for s in scores[1]:
                    # Multiple values (for each split) for
                    # each (hidden_dim, n_layer) combination
                    hidden_dims_data.append(hidden_dim)
                    hidden_layers_data.append(n_layer)
                    scores_data.append(s)
                    
                mean = np.mean(scores[1])
                var = np.var(scores[1])

        ax.set_xlabel('hidden layer units')
        ax.set_ylabel('number of hidden layers')
        
        #if metric_name in ('rmse_mc', 'rmse_non_mc'):
        #    ax.set_zlim([0, 5])
        #elif metric_name == 'lls_mc':
        #    ax.set_zlim([-5, 0])
    
        ax.scatter3D(hidden_dims_data, hidden_layers_data, scores_data, c=scores_data)

        fig.suptitle(figure_title, y=2.05)        
        
        plt.savefig(
            os.path.join(experiment_root_directory, figure_file_name),
            dpi=300,
            bbox_inches='tight',
        )
        
        plt.show()